In [1]:
import pandas as pd

import glob
import json

import datetime
from datetime import datetime
from datetime import timedelta

In [2]:
data_files = sorted(glob.glob('../Training Data/Kickstarter*.csv'))
df = pd.concat((pd.read_csv(file) for file in data_files))

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# Timestamps were in the form of seconds elapsed since origin, so converted to create new dates

def clean_date(column):
    date_list = []
    start_date = datetime(year=1970, month=1, day=1)
    for r in column:
        new_date = start_date + timedelta(seconds = int(r))
        date_list.append(new_date)
    return date_list

In [5]:
# Parses columns that are nested jsons from data source to extract a certain feature.A
# Accepts a JSON column from the csv (column) and a keyword to extract from that column (keyword)

def parse_json(column, keyword):
    new_column = []
    for r in column:
        try:
            new_column.append(json.loads(r)[keyword])
        except:
            new_column.append(None)
    return new_column

In [6]:
df.created_at = clean_date(df.created_at)
df.state_changed_at = clean_date(df.state_changed_at)
df.deadline = clean_date(df.deadline)
df.launched_at = clean_date(df.launched_at)

In [7]:
df.category = parse_json(df.category, 'name')
df.creator = parse_json(df.creator, 'name')
df['location_city'] = parse_json(df.location, 'localized_name')
df['location_state'] = parse_json(df.location, 'state')

In [8]:
# drops duplicate rows that match on id, deadline, and creator
# PHIL: Document why uniqueness is expected at this level
df = df.drop_duplicates(subset=['id', 'deadline', 'creator'])

In [9]:
# drop json Array columns
df = df.drop(columns=['location','photo','profile','urls'])

In [10]:
df = df.reset_index().drop(columns='index')

In [11]:
# save to pickle
df.to_pickle('../Tables/loaded_data.pkl')

In [12]:
# save off sepearate file
df.to_csv('../Tables/loaded_data.csv')